In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

In [6]:
llm = ChatOpenAI()


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)


# txt,doc,pdf.html,excel 등등 다양한 file들을 load 할 수 있다.
loader = UnstructuredFileLoader("../KAIA-JKIICE.pdf")

docs = loader.load_and_split(text_splitter=splitter)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dahun\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dahun\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
Created a chunk of size 1029, which is longer than the specified 600
Created a chunk of size 864, which is longer than the specified 600
Created a chunk of size 643, which is longer than the specified 600
Created a chunk of size 636, which is longer than the specified 600
Created a chunk of size 726, which is longer than the specified 600
Created a chunk of size 918, which is longer than the specified 600
Created a chunk of size 692, which is longer than the specified 600
Created a chunk of size 1431, which is longer than the specified 600


In [7]:
docs

[Document(page_content='JKIICE Journal of the Korea Institute of Information and\nCommunication Engineering\n한국정보통신학회논문지 Vol. 23, No. 1: 399~406, Mar. 2019\nYOLO 기반 딥러닝 알고리즘을 이용한 도로 장애물 탐지 및 내비게이션 연동 시스템\nRoad Obstacle Detection and Navigation YOLO-based Deep Learning Algorithm\nInterworking System Using\n요 약', metadata={'source': '../KAIA-JKIICE.pdf'}),
 Document(page_content='기존의 도로 안전 시스템들은 다양한 도로 상황과 환경적 변화에 능동적으로 대응하는 데 한계를 가지고 있으며, 대규모 인프라 투자의 필요성과 상당한 시간과 노력이 소요되는 문제점을 가지고 있다. 본 논문은 이러한 기존 기술 들의 한계를 극복하기 위하여, YOLO (You Only Look Once) 기반의 딥러닝 알고리즘과 첨단 센서 기술을 결합함으 로써, 교통 장애 요인을 실시간으로 정확하게 탐지하고 이에 빠르게 대응할 수 있는 시스템을 제안하였다. 열화상 카 메라와 고해상도 광각 카메라를 이용하여 도로상의 장애물을 탐지하고, 공개 데이터 소스로부터 추가 데이터를 확보 하여 데이터 셋의 다양성과 품질을 보장하였으며, 네비게이션 시스템과의 연동을 통해 도로상의 장애물 위치 등을 지도상 해당 좌표에 도로 상태를 나타내도록 하였다. 실험을 통하여 개발된 YOLO 기반 모델은 다양한 조명과 환경 조건에서도 블랙아이스, 포트홀 및 장애물을 실시간으로 분석하고 높은 정확도로 식별할 수 있음을 보였다.', metadata={'source': '../KAIA-JKIICE.pdf'}),
 Document(page_content='ABSTRACT\nExisting road safety systems have 

In [10]:
# embedding은 고ㅇ

cache_dir = LocalFileStore("./.cache/")

# Embedding은 text에 의미별로 적절한 점수를 부여해서 vector 형식으로 표현한것,

embeddings = OpenAIEmbeddings()

# ChacheBackedEmbedding을 사용하여 만들어진 Embedding을 cache(저장) 한다.

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings,cache_dir)

# Vector store를 호출 해주고 FAISS사용
# from_documents 메서드는 document 별로 embedding 작업 후 결과를 저장한 vector store를 반환
# 이를 이용해 document 검색도 하고, 연관성이 높은 document들을 찾는다.
vectorstore = FAISS.from_documents(docs,cached_embeddings)

# retriever은 LangChain이 제공하는 class 또는 interface의 일종이다.
# document를 검색해서 찾아오는 기능을 가지고 있다.
# map_rerank는 각 document에 기반해서 질문에 대답하고 답변에 점수를 매겨준다,
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",
    retriever=vectorstore.as_retriever(),
)

chain.run("이건 무엇에 관한 논문이니? 한글로 말해줘")

c:\Users\dahun\Desktop\fullstack-gpt\env\Lib\site-packages\langchain\chains\llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'This document discusses the limitations of existing research focused on fixed sensor systems or limited ranges in adapting to various road situations and environmental changes, as well as the problems of the necessity of large-scale infrastructure investment and significant time and effort required.'